### Aceleração para falha na pista externa

In [ ]:
import numpy as np

def calculate_outer_race_acceleration(coeffs, df_modes, duration_sec=0.1, fs_hz=12000):
    """
    Calcula o sinal de ACELERAÇÃO (m/s²) para defeito na Pista Externa (Outer Race),
    baseado na Eq. 19 de Tandon & Choudhury.
    
    A aceleração amplifica as altas frequências, multiplicando a amplitude de
    deslocamento por -(omega^2).
    """
    t = np.arange(0, duration_sec, 1/fs_hz)
    acceleration = np.zeros_like(t)
    
    # 1. Carregar Parâmetros
    # P_max: Carga no centro da zona (xi=0)
    # Assume-se que o defeito está fixo na zona de carga máxima (pior caso)
    P_max = coeffs['debug_info']['P_max_N']      
    Z = 9
    
    # Frequência de Defeito (rad/s) - BPFO (Ball Pass Frequency Outer)
    f_defect_hz = coeffs['frequencies']['defect_freq_hz']
    w_defect = 2 * np.pi * f_defect_hz
    
    # 2. Calcular Compliância Estática (O termo escalar)
    # Representa a flexibilidade da estrutura (soma de 1/rigidez dinâmica dos modos)
    structural_compliance = 0.0
    df_outer = df_modes[df_modes['Race'] == 'Outer']
    
    for _, row in df_outer.iterrows():
        w_i = row['Freq_rad_s']
        M_i = row['Mass_kg']
        structural_compliance += 1.0 / (M_i * (w_i**2))
        
    # Amplitude Base de Deslocamento (Estática)
    static_amplitude = P_max * structural_compliance

    # 3. Loop de Somatório dos Harmônicos (Eq 19 derivada)
    # a(t) = d²y/dt²
    
    N_harmonics = 50 
    
    for s in range(1, N_harmonics + 1):
        # Frequência angular deste harmônico
        w_s = s * w_defect
        
        # Coeficiente do Pulso F no índice Z*s
        harmonic_index = int(Z * s)
        F_Zs = coeffs['Fs'](harmonic_index)
        
        # Amplitude de Deslocamento deste harmônico (sem a parte temporal)
        disp_amp_component = static_amplitude * (Z * F_Zs)
        
        # --- APLICAÇÃO DA ACELERAÇÃO ---
        # Multiplica por -(omega^2)
        acc_component = -(w_s**2) * disp_amp_component
        
        # Adiciona ao sinal
        acceleration += acc_component * np.cos(w_s * t)
        
    return t, acceleration

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Parameters
rpm_val = 1730
fault_mm = 0.178  # Approx 0.007 inches
fault_type = 'outer'

# Calculation
df_modes = get_bearing_natural_frequencies()
coeffs = calculate_tandon_coefficients(fault_mm, rpm_val, fault_type)
t, signal = calculate_outer_race_acceleration(coeffs, df_modes, duration_sec=0.1)

# Plot
plt.figure(figsize=(15, 5))
plt.plot(t, signal, color='red')
plt.title(f'Synthetic Outer Race Defect Signal\nRPM: {rpm_val} | Fault: {fault_mm}mm')
plt.xlabel('Time (s)')
plt.ylabel('Displacement (m/s)')
plt.grid(True, alpha=0.3)
plt.show()

### Aceleração para falha na pista interna

In [ ]:
import numpy as np

def calculate_inner_race_acceleration(
    coeffs, 
    df_modes, 
    duration_sec=1.0, 
    fs_hz=12000, 
    N_harmonics_defect=30,  
    M_harmonics_sideband=7, 
    S_harmonics_pulse=15
):
    """
    Calcula a ACELERAÇÃO a(t) para falha na PISTA INTERNA.
    Matematicamente: a(t) = d²y/dt² -> Multiplica cada termo espectral por -(omega^2).
    """
    
    # 1. Setup
    t = np.arange(0, duration_sec, 1/fs_hz)
    acc_time = np.zeros_like(t) # Sinal de aceleração
    
    # Constantes e Frequências
    Z = 9
    f_defect = coeffs['frequencies']['defect_freq_hz']
    f_shaft = coeffs['frequencies']['shaft_freq_hz']
    w_id = 2 * np.pi * f_defect
    w_s  = 2 * np.pi * f_shaft
    
    Po = coeffs['Po']
    Fo = coeffs['Fo']
    get_Pr = coeffs['Pr']
    get_Fs = coeffs['Fs']

    # 2. Compliância (Mantida igual - define a magnitude do deslocamento base)
    structural_compliance = 0.0
    df_outer = df_modes[df_modes['Race'] == 'Outer']
    if df_outer.empty:
        structural_compliance = 1.0e-8 
    else:
        for _, row in df_outer.iterrows():
            structural_compliance += 1.0 / (row['Mass_kg'] * (row['Freq_rad_s']**2))

    # 3. Síntese da Aceleração
    for j in range(1, N_harmonics_defect + 1):
        w_carrier = j * w_id
        
        # --- A. Portadora ---
        term_a1 = Z * Po * Fo * structural_compliance
        term_a2 = 0.0
        for s in range(1, S_harmonics_pulse + 1):
            Fs = get_Fs(s)
            term_a2 += (Z * Po * Fs / 2.0) * structural_compliance
        
        A_j = term_a1 + term_a2
        
        # *** MUDANÇA AQUI: Converter Deslocamento -> Aceleração ***
        # a(t) = -w² * A * cos(wt)
        acc_time += - (w_carrier**2) * A_j * np.cos(w_carrier * t)
        
        # --- B. Bandas Laterais ---
        for r in range(1, M_harmonics_sideband + 1):
            # Frequências
            w_sb_upper = w_carrier + (r * w_s)
            w_sb_lower = np.abs(w_carrier - (r * w_s))
            
            Pr = get_Pr(r)
            
            # Amplitude de Deslocamento (B_jr)
            term_b1 = (Z * Pr * Fo / 2.0) * structural_compliance
            term_b2 = 0.0
            for s in range(1, S_harmonics_pulse + 1):
                Fs = get_Fs(s)
                term_b2 += (Z * Pr * Fs / 4.0) * structural_compliance
            B_jr = term_b1 + term_b2
            
            # *** MUDANÇA AQUI: Multiplica cada banda por sua frequência ao quadrado ***
            acc_time += - (w_sb_upper**2) * B_jr * np.cos(w_sb_upper * t)
            acc_time += - (w_sb_lower**2) * B_jr * np.cos(w_sb_lower * t)

    return t, acc_time

In [ ]:
rpm = 1730
fault_mm = 0.178 # 0.007"

# 1. Calcula modos e coeficientes
df_modes = get_bearing_natural_frequencies()
coeffs = calculate_tandon_coefficients(fault_mm, rpm, 'inner')

# 2. CHAMA A FUNÇÃO (Gera sinal de Deslocamento)
t, y_disp = calculate_inner_race_acceleration(coeffs, df_modes)

# 3. Plotagem
plt.figure(figsize=(15, 5))
plt.plot(t, y_disp * 1e6, color='purple', linewidth=1) # Em micrômetros
plt.title(f"Sinal Sintético (Função Original do Notebook): Pista Interna\nRPM: {rpm} | Defeito: {fault_mm}mm")
plt.xlabel("Tempo (s)")
plt.ylabel("Deslocamento (µm)")
plt.grid(True, alpha=0.3)
plt.show()